In [32]:
# Importing neccessary libraries and packages
import os
from dotenv import load_dotenv
from langchain_community.tools import TavilySearchResults,DuckDuckGoSearchRun
import yfinance as yf
from langchain_core.tools import tool
from pydantic import BaseModel, Field
from langgraph.graph import Graph,StateGraph,MessagesState,END,START
from langgraph.checkpoint import memory
from langgraph.prebuilt import ToolNode,tools_condition
from langchain_core.messages import HumanMessage,SystemMessage
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
from typing import TypedDict,Annotated
from langchain_core.messages import AnyMessage
import operator
load_dotenv()

True

In [33]:
# Loading the environment variables
# Setting the environment variables
os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv("HUGGINGFACEHUB_API_TOKEN")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["MISTRAL_API_KEY"] = os.getenv("MISTRAL_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

In [42]:
# Initializing the LLM model
llm = ChatGroq(model="llama-3.3-70b-versatile",temperature=0)
llm=ChatGroq(model_name="Gemma2-9b-It")

In [43]:
search = DuckDuckGoSearchRun()

In [44]:
def get_stock_price(ticker: str) -> float:
    """Gets a stock price from Yahoo Finance.

    Args:
        ticker: ticker str
    """
    # """This is a tool for getting the price of a stock when passed a ticker symbol"""
    stock = yf.Ticker(ticker)
    return stock.info['previousClose']

In [45]:
tools = [search,get_stock_price]
llm_with_tools=llm.bind_tools(tools)

In [46]:
class GraphState(TypedDict):
    """State of the graph."""
    query: str
    finance: str
    final_answer: str
    # intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]
    messages: Annotated[list[AnyMessage], operator.add]

In [47]:
def reasoner(state):
    query = state["query"]
    messages = state["messages"]
    # System message
    sys_msg = SystemMessage(content="You are a helpful assistant tasked with using search, the yahoo finance tool")
    message = HumanMessage(content=query)
    messages.append(message)
    result = [llm_with_tools.invoke([sys_msg] + messages)]
    return {"messages":result}

In [48]:
workflow = StateGraph(GraphState)

# Add Nodes
workflow.add_node("reasoner", reasoner)
workflow.add_node("tools", ToolNode(tools)) # for the tools

# Add Edges
workflow.add_edge(START, "reasoner")

workflow.add_conditional_edges(
    "reasoner",
    # If the latest message (result) from node reasoner is a tool call -> tools_condition routes to tools
    # If the latest message (result) from node reasoner is a not a tool call -> tools_condition routes to END
    tools_condition,
)
workflow.add_edge("tools", "reasoner")
react_graph = workflow.compile()

In [49]:
response = react_graph.invoke({"query": "What is price of TCS", "messages": []})
for m in response['messages']:
    m.pretty_print()

================================ Human Message =================================

What is price of TCS
================================== Ai Message ==================================
Tool Calls:
  get_stock_price (call_jtaa)
 Call ID: call_jtaa
  Args:
    ticker: TCS
================================= Tool Message =================================
Name: get_stock_price

2.16
================================ Human Message =================================

What is price of TCS
================================== Ai Message ==================================

2.16
